In [ ]:
# Проверка, что среда Colab активна и доступен GPU
!nvidia-smi
import torch

print("torch:", torch.__version__, "| CUDA доступна:", torch.cuda.is_available())

In [ ]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

In [ ]:
import os

BASE = "/content/drive/MyDrive/llm-news"
for sub in ["models", "metrics", "hf_cache"]:
    os.makedirs(os.path.join(BASE, sub), exist_ok=True)

print("Созданы/проверены папки:", os.listdir(BASE))

In [ ]:
import subprocess
import sys

REPO_URL = "https://github.com/mdayssi/llm-news-summarizer-ru.git"
REPO_DIR = "/content/llm-news"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    print("Репозиторий уже есть:", REPO_DIR)

# Дальше будем работать из корня репо
%cd {REPO_DIR}
!git rev-parse --short HEAD

In [ ]:
from pathlib import Path

env_path = Path(REPO_DIR) / ".env"
kv = {
    "EXTERNAL_MODELS_DIR": "/content/drive/MyDrive/llm-news/models",
    "EXTERNAL_METRICS_DIR": "/content/drive/MyDrive/llm-news/metrics_big",
    "EXTERNAL_CACHE_DIR": "/content/drive/MyDrive/llm-news/hf_cache",
}
text = "\n".join([f"{k}={v}" for k, v in kv.items()]) + "\n"
env_path.write_text(text, encoding="utf-8")

print(".env создано:")
print(env_path.read_text())

In [ ]:
%pip -q install --upgrade \
  evaluate rouge-score \
  razdel bitsandbytes\
  python-dotenv pyyaml \

import datasets
import torch
import transformers
import accelerate
import rouge_score
import evaluate
import sentencepiece
import razdel
import dotenv
import yaml
import tqdm
import bitsandbytes

print("torch:", torch.__version__, "| cuda avail:", torch.cuda.is_available())
print("transformers:", transformers.__version__)
print("datasets:", datasets.__version__)
print("evaluate:", evaluate.__version__)

In [ ]:
import sys

repo_src = "/content/llm-news/src"
if repo_src not in sys.path:
    sys.path.insert(0, repo_src)
print("sys.path ok")